In [2]:
import networkx as nx
import random
import numpy as np
import pickle
import matplotlib.pyplot as plt
from time import time

# Datasets

In [3]:
#Facebook Dataset
dir = "C:/Users/ruiji/Documents/USC/Research/PHY760/Data/Social Networks/Facebook/"

In [4]:
GraphType = nx.Graph()

In [5]:
FB_Net = nx.read_edgelist(
    dir+"facebook_combined.txt", 
    create_using=GraphType,
    nodetype=int
)

# Func

In [6]:

  
  
def timer_func(func):
    # This function shows the execution time of 
    # the function object passed
    def wrap_func(*args, **kwargs):
        t1 = time()
        result = func(*args, **kwargs)
        t2 = time()
        print(f'Function {func.__name__!r} executed in {(t2-t1):.4f}s')
        return result
    return wrap_func

In [7]:
def gen_rand_subgraphs(graph,N=10,size=200):
    result = []
    random_source_nodes = random.sample(graph.nodes,N)
    for node in random_source_nodes:
        remain_nodes = list(nx.shortest_path(graph, source=node).keys())[:size]
        subgraph = graph.subgraph(remain_nodes).copy()
        result.append(subgraph)
    return result

In [8]:
def plot(map_result,keywords_list,m=2,annotate=True,node=None,membership='blue'):
    ax_min = np.min(map_result)*1.2
    ax_max = np.max(map_result)*1.2
    fig = plt.figure(figsize=(12,10))
    if m == 2:
        ax = fig.add_subplot()
        ax.scatter(map_result[:,0], map_result[:,1],c=membership)
        ax.set_xlim(ax_min,ax_max)
        ax.set_ylim(ax_min,ax_max)
        if annotate:
            for (label,x,y) in zip(keywords_list,map_result[:,0],map_result[:,1]):
                plt.annotate(label, (x, y))
        if node:
            id = keywords_list.index(node)
            plt.scatter(map_result[id,0],map_result[id,1],color="red")
            plt.annotate(node, (map_result[id,0], map_result[id,1]))
        plt.show
    else:
        ax = plt.axes(projection='3d')
        ax.scatter3D(map_result[:,0], map_result[:,1], map_result[:,2])
        ax.set_xlim(ax_min,ax_max)
        ax.set_ylim(ax_min,ax_max)
        ax.set_zlim(ax_min,ax_max)
        if node:
            id = keywords_list.index(node)
            ax.scatter3D(map_result[id,0],map_result[id,1],map_result[id,2],color="red")
        '''for (label,x,y,z) in zip(keywords_list,map_result[:,0],map_result[:,1],map_result[:,2]):
            plt.annotate(label, (x,y,z))'''
        plt.show

In [9]:
import fastmap_graphs as fmg
from dynFM import compute_rotation_matrix

# Patching

In [11]:
import gurobipy as gp
from gurobipy import GRB

In [10]:
import importlib
importlib.reload(fmg)

<module 'fastmap_graphs' from 'c:\\Users\\ruiji\\Documents\\USC\\Research\\PHY760\\Notes\\fastmap_graphs.py'>

In [12]:
def Gurobi_Patch_Solver(Z0,X1):
    K = Z0.shape[1]
    N = Z0.shape[0]
    m = gp.Model("Patching") # Create Model
    m.params.NonConvex = 2
    m.params.LogToConsole = 0
    A = m.addMVar((K,K))
    b = m.addMVar(K)
    Y = m.addMVar((N,K), lb=-GRB.INFINITY, name='Y')
    m.addConstrs((Y[j] == sum([X1[j][i] * A[:,i] for i in range(K)])+b-Z0[j] for j in range(N)), name='set_Y')
    m.addConstrs((A[:,i]@A[:,i]==1 for i in range(K)), name='set A linear trans')
    for i in range(K):
        for j in range(i+1,K):
            m.addConstr((A[:,i]@A[:,j]==0), name=f'set A orthogonal_{i}_{j}')
    m.update()
    #m.setObjective(sum([Y[i,:] @ Y[i,:] for i in range(N)]), GRB.MINIMIZE)
    obj = sum(Y[i] @ Y[i] for i in range(N))
    m.setObjective(obj, GRB.MINIMIZE)
    m.optimize()
    beta = np.mean(Z0-X1,axis = 0)
    print(beta)
    Z1 = X1@A.X+beta
    return Z1

In [13]:
K, e, T, M, isBM, times = 3, 0.001, 10, 4, True, 10
k_clusters = 3
time_measure = {"params":{"K":K, "e":e, "T":T, "M":M, "isBM":isBM, "times":times}}
for size in [200,500,1000,2000,4000]:
    time_measure[size] = {}
    test_nets = gen_rand_subgraphs(FB_Net,N=3,size=size)
    with open(f'../Data/FMBM_size{size}_net.pickle','wb') as handle:
        pickle.dump(test_nets,handle)

    ## FMBM
    t_cost_sum = []
    fm_embeddings = []
    for test_net in test_nets:
        test_adj_mat = nx.adjacency_matrix(test_net)
        t_start = time()
        labels, score, runtime, G, fm_embedding = fmg.bestscore_choice(test_adj_mat, K, e, k_clusters, T, times, isBM)
        t_cost = time()-t_start
        t_cost_sum.append(t_cost)
        print(f"***FMBM cost for {size}: {t_cost}")
        fm_embeddings.append(fm_embedding)
    time_measure[size]["fmbm"] = sum(t_cost_sum)/len(t_cost_sum)
    with open(f'../Data/FMBM_size{size}_emb.pickle','wb') as handle:
        pickle.dump(fm_embeddings,handle)
    
    ## Patching
    t_cost_sum = []
    Z0 = fm_embeddings[0]
    patched_subnets = [fm_embeddings[0]]
    for X1 in fm_embeddings[1:]:
        t_start = time()
        Z1 = Gurobi_Patch_Solver(Z0,X1)
        t_cost = time()-t_start
        t_cost_sum.append(t_cost)
        print(f"***DFM cost for {size}: {t_cost}")
        patched_subnets.append(Z1)
        Z0 = Z1
    time_measure[size]["dfm"] = sum(t_cost_sum)/len(t_cost_sum)
    with open(f'../Data/DFM_size{size}_emb.pickle','wb') as handle:
        pickle.dump(patched_subnets,handle)

with open(f'../Data/time_measure.pickle','wb') as handle:
        pickle.dump(time_measure,handle)

C:\Users\ruiji\AppData\Local\Temp/ipykernel_23628/1179480307.py:3: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  random_source_nodes = random.sample(graph.nodes,N)


***FMBM cost for 200: 152.59118628501892
***FMBM cost for 200: 164.0543191432953
***FMBM cost for 200: 100.26970028877258
Set parameter Username
Academic license - for non-commercial use only - expires 2023-06-30
Set parameter NonConvex to value 2
[-443.78803904 1290.45716265 -473.40424169]
***DFM cost for 200: 0.9171020984649658
Set parameter NonConvex to value 2
[ -298.96404816  2633.04859023 -1854.17876608]
***DFM cost for 200: 0.7400736808776855


C:\Users\ruiji\AppData\Local\Temp/ipykernel_23628/1179480307.py:3: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  random_source_nodes = random.sample(graph.nodes,N)


***FMBM cost for 500: 295.68598771095276
***FMBM cost for 500: 155.03143572807312
***FMBM cost for 500: 387.6433403491974
Set parameter NonConvex to value 2
[-436.99070392 -307.30085804 -441.66385028]
***DFM cost for 500: 2.246034622192383
Set parameter NonConvex to value 2
[-369.38703929 -187.35771771 -192.78658917]
***DFM cost for 500: 2.2099626064300537


C:\Users\ruiji\AppData\Local\Temp/ipykernel_23628/1179480307.py:3: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  random_source_nodes = random.sample(graph.nodes,N)


***FMBM cost for 1000: 495.8780519962311
***FMBM cost for 1000: 820.4001131057739
***FMBM cost for 1000: 955.609071969986
Set parameter NonConvex to value 2
[ 50.41887623 153.07840649 625.09413617]
***DFM cost for 1000: 4.4975831508636475
Set parameter NonConvex to value 2
[-143.48112228   35.43000852  394.6553152 ]
***DFM cost for 1000: 4.130925416946411


C:\Users\ruiji\AppData\Local\Temp/ipykernel_23628/1179480307.py:3: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  random_source_nodes = random.sample(graph.nodes,N)


***FMBM cost for 2000: 2070.6970884799957
***FMBM cost for 2000: 1957.7727122306824
***FMBM cost for 2000: 1775.5956885814667
Set parameter NonConvex to value 2
[-130.6103995   119.40650548   71.16243204]
***DFM cost for 2000: 11.905152082443237
Set parameter NonConvex to value 2
[-179.61450189  -41.00624306  239.8204935 ]
***DFM cost for 2000: 10.826496362686157


C:\Users\ruiji\AppData\Local\Temp/ipykernel_23628/1179480307.py:3: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  random_source_nodes = random.sample(graph.nodes,N)


***FMBM cost for 4000: 5668.593617200851
***FMBM cost for 4000: 5729.245294570923
***FMBM cost for 4000: 5905.7825264930725
Set parameter NonConvex to value 2
[  98.59028252 -243.87323069   13.32148079]
***DFM cost for 4000: 36.63773202896118
Set parameter NonConvex to value 2
[ 229.73422564 -349.05372879 -222.08134322]
***DFM cost for 4000: 30.658745765686035


In [15]:
time_measure["params"] = {"K":K, "e":e, "T":T, "M":M, "isBM":isBM, "times":times}
with open(f'../Data/time_measure.pickle','wb') as handle:
        pickle.dump(time_measure,handle)

In [16]:
time_measure

{200: {'fmbm': 138.97173523902893, 'dfm': 0.8285878896713257},
 500: {'fmbm': 279.4535879294078, 'dfm': 2.2279986143112183},
 1000: {'fmbm': 757.2957456906637, 'dfm': 4.314254283905029},
 2000: {'fmbm': 1934.6884964307148, 'dfm': 11.365824222564697},
 4000: {'fmbm': 5767.873812754949, 'dfm': 33.64823889732361},
 'params': {'K': 3, 'e': 0.001, 'T': 10, 'M': 4, 'isBM': True, 'times': 10}}